In [33]:
import numpy as np
import os
import tensorflow as tf
%load_ext autoreload
%autoreload 2
%pylab inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Populating the interactive namespace from numpy and matplotlib


/home/bzr0014/anaconda3/envs/tensorGpu/lib/python3.6/site-packages/IPython/core/magics/pylab.py:160: UserWarning: pylab import has clobbered these variables: ['mean', 'sum']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [34]:
filename = 'odds.csv'
oddsFile = open(filename, 'r')
oddsFile.readline()
indexDict = {}
countryDict = {}
odds = []
probs = []
accumOdds = []
sum = 0
for line in oddsFile:
    index, country, odd, prob = line.split(',')
    indexDict[int(index)-1] = country
    countryDict[country] = int(index)-1
    odds.append(float(odd))
    probs.append(float(prob))
    accumOdds.append(sum)
    sum += float(prob)
accumOdds.append(sum)
odds = np.array(odds)
probs = np.array(probs)
accumOdds = np.array(accumOdds) / np.max(accumOdds)
#probs = 1 / odds
#probs = probs / np.sum(probs)
#for i in range(1, 33):
#    accumOdds[i] = accumOdds[i-1] + probs[i-1]
print(accumOdds)
print(probs)

[0.         0.12868949 0.12868949 0.17355372 0.34120425 0.37072019
 0.37072019 0.39197166 0.40613932 0.4214876  0.44391972 0.56788666
 0.70247934 0.71428571 0.71428571 0.71428571 0.71428571 0.73435655
 0.74025974 0.74025974 0.74025974 0.74025974 0.7579693  0.79693034
 0.81700118 0.81700118 0.81700118 0.81700118 0.92680047 0.94805195
 0.96458087 0.96458087 1.        ]
[10.9  0.   3.8 14.2  2.5  0.   1.8  1.2  1.3  1.9 10.5 11.4  1.   0.
  0.   0.   1.7  0.5  0.   0.   0.   1.5  3.3  1.7  0.   0.   0.   9.3
  1.8  1.4  0.   3. ]


In [35]:
def rel_error(x, y):
    return np.max(np.abs(x - y) / (np.maximum(1e-8, np.abs(x) + np.abs(y))))

def create_scenario(probs, N):
    K = len(probs)-1
    probs = (probs * N).astype('uint32')
    scenarios = np.zeros((N, K))
    count = 0
    for probIndex in range(1, len(probs)):
        while count < N and count < probs[probIndex]:
            scenarios[count, probIndex-1] = 1
            count += 1
    return scenarios

In [59]:
tf.reset_default_graph()
stdSize = .5
k = len(probs)
init = tf.constant(np.zeros((1, k), dtype='float32'))
odds_tf = tf.placeholder(tf.float32, [1, k])
#odds_tf = tf.constant(odds.reshape(1, k).astype('float32'))
scenarios_tf = tf.placeholder(tf.float32, [None, k])
#scenarios_tf = tf.constant(scenarios.astype('float32'))
def myModel(odds, scenarios, stdSize):
    bets = tf.get_variable('bets', [1, k])+2
    h0 = tf.nn.relu(bets)
    #h0 = tf.abs(bets)
    h1 = tf.divide(h0, tf.reduce_sum(h0)+1e-9)
    N = tf.shape(scenarios)[0]
    ones = tf.ones([N, 1])
    bets_reshaped = tf.matmul(ones, h1)
    odds_reshaped = tf.matmul(ones, odds)
    allResults = tf.subtract(tf.multiply(tf.multiply(bets_reshaped, odds_reshaped), scenarios), bets_reshaped)
    wins = tf.reduce_sum(allResults, axis=1)
    return wins, h1
win_tf, bets_tf = myModel(odds_tf, scenarios_tf, stdSize)
#mean, var = tf.nn.moments(win_tf, axes=[0])
#loss = -mean + 0 * tf.sqrt(var)
mean = tf.reduce_mean(win_tf)
loss = -mean + stdSize * tf.sqrt(tf.reduce_mean(tf.pow(win_tf - mean, 2)))

In [60]:
optimizer = tf.train.AdamOptimizer(1e-2)
#optimizer = tf.train.RMSPropOptimizer(1e-4)
train_step = optimizer.minimize(loss)
#train_step = tf.train.GradientDescentOptimizer(1e-3).minimize(loss)

In [61]:
N = 1000
scenarios = create_scenario(accumOdds, N)
feed_dict = {odds_tf:odds.reshape(1, k), scenarios_tf:scenarios}
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    myLoss, bets,_ = sess.run([loss, bets_tf, train_step], feed_dict=feed_dict)
    #myLoss, bets,_ = sess.run([loss, bets_tf, train_step])
print(bets)
betsReshaped = np.ones((N, 1)).dot(bets)
oddsReshaped = np.ones((N, 1)).dot(odds.reshape(1, -1))
allResults = betsReshaped * oddsReshaped * scenarios - betsReshaped
wins = np.sum(allResults, axis=1)
print(wins.shape)
print(rel_error(np.mean(wins), -myLoss))
print(rel_error(np.mean(wins)-stdSize*np.std(wins), -myLoss))
print("Average wins:", -myLoss)

[[0.02504113 0.03296627 0.03679257 0.03220965 0.03238441 0.03279231
  0.03202727 0.02866492 0.03261162 0.02535311 0.03072651 0.03305328
  0.03372848 0.03281198 0.02551173 0.02440586 0.03324769 0.02757432
  0.03630212 0.03334066 0.03689581 0.02456594 0.0294775  0.03269244
  0.03461358 0.02976711 0.03067952 0.02694684 0.0358285  0.03425959
  0.02782012 0.03490716]]
(1000,)
0.7201921327708896
1.3286205587049318e-08
Average wins: -0.9890734


In [ ]:
N = 1000
scenarios = create_scenario(accumOdds, N)
feed_dict = {odds_tf:odds.reshape(1, k), scenarios_tf:scenarios}
num_epochs = 3000
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(num_epochs):
        myLoss, bets, _ = sess.run([loss, bets_tf, train_step], feed_dict={odds_tf:odds.reshape(1, k), scenarios_tf:scenarios})
        #myLoss, bets, _ = sess.run([loss, bets_tf, train_step])
        if (i % 1000 == 0):
            print(-myLoss)        #bets, myLoss,_ = sess.run([bets_tf, loss, train_step])
print(bets)
betsReshaped = np.ones((N, 1)).dot(bets)
oddsReshaped = np.ones((N, 1)).dot(odds.reshape(1, -1))
allResults = betsReshaped * oddsReshaped * scenarios - betsReshaped
wins = np.sum(allResults, axis=1)
print(wins.shape)
print(rel_error(np.mean(wins), -myLoss))
print(rel_error(np.mean(wins)-stdSize*np.std(wins), -myLoss))
print("Average wins:", -myLoss)

In [57]:
for i in range(len(bets[0])):
    print("%14s: %4.2f"%(indexDict[i], bets[0, i]*100))

     argentina: 0.00
     australia: 0.00
       belgium: 0.00
        brazil: 0.00
      colombia: 0.00
    costa rica: 0.00
       croatia: 0.00
       denmark: 0.00
         egypt: 19.24
       england: 0.00
        france: 0.00
       germany: 0.00
       iceland: 16.74
          iran: 0.00
         japan: 0.00
         korea: 0.00
        mexico: 19.61
       morocco: 9.47
       nigeria: 0.00
        panama: 0.00
          peru: 0.00
        poland: 0.00
      portugal: 0.00
        russia: 0.00
  saudi arabia: 0.00
       senegal: 0.00
        serbia: 0.00
         spain: 0.00
        sweden: 34.94
   switzerland: 0.00
       tunisia: 0.00
       uruguay: 0.00


In [58]:
bets_new = bets * 1
index1 = countryDict['australia']
index2 = countryDict['argentina']
#bets_new[0, index1] -= .1
#bets_new[0, index2] += .1
print(index)
#bets_new[0, index] = 100
betsReshaped = np.ones((N, 1)).dot(bets_new)
oddsReshaped = np.ones((N, 1)).dot(odds.reshape(1, -1))
allResults = betsReshaped * oddsReshaped * scenarios - betsReshaped
wins = np.sum(allResults, axis=1)
print(wins.shape)
print("relative error 1 -> ", rel_error(np.mean(wins), -myLoss))
print("relative error 2 -> ", rel_error(np.mean(wins)-stdSize*np.std(wins), -myLoss))
print("Average wins:", wins.mean())
print("std wins:", wins.std())
print("Average wins min %2.1f standard deviation:"%stdSize, np.mean(wins)-stdSize*np.std(wins))

32
(1000,)
relative error 1 ->  0.43085978993938223
relative error 2 ->  7.899037545890518e-08
Average wins: 1.6640122950077056
std wins: 10.021330085816498
Average wins min 0.1 standard deviation: 0.6618792864260559
